## capture and  Store the image data (Run this code first )

In [ ]:
import cv2, os
haar_file = 'haarcascade_frontalface_default.xml'
datasets = 'Dataset'  
sub_data = 'hari'     

path = os.path.join(datasets, sub_data)
if not os.path.isdir(path):
    os.mkdir(path)
(width, height) = (130, 100)   


face_cascade = cv2.CascadeClassifier(haar_file)
webcam = cv2.VideoCapture(0) 

count = 1
while count < 31:
    print(count)
    (_, im) = webcam.read()
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 4)
    for (x,y,w,h) in faces:
        cv2.rectangle(im,(x,y),(x+w,y+h),(255,0,0),2)
        face = gray[y:y + h, x:x + w]
        face_resize = cv2.resize(face, (width, height))
        cv2.imwrite('%s/%s.png' % (path,count), face_resize)
        count += 1
        
    cv2.imshow('OpenCV', im)
    key = cv2.waitKey(10)
    if key == 27:
        break
webcam.release()
cv2.destroyAllWindows()


## Recognizing the face(Run this code as the second part to identify the person)

In [4]:
import cv2, numpy, os
haar_file = 'haarcascade_frontalface_default.xml'
face_cascade = cv2.CascadeClassifier(haar_file)
datasets = 'Dataset'
print('Training...')
(images, labels, names, id) = ([], [], {}, 0)

for (subdirs, dirs, files) in os.walk(datasets):
    for subdir in dirs:
        names[id] = subdir
        subjectpath = os.path.join(datasets, subdir)
        for filename in os.listdir(subjectpath):
            path = subjectpath + '/' + filename
            label = id
            images.append(cv2.imread(path, 0))
            labels.append(int(label))
        id +=1

(images, labels) = [numpy.array(lis) for lis in [images, labels]]
print(images, labels)                   
(width, height) = (130, 100)
model = cv2.face.LBPHFaceRecognizer_create()
#model =  cv2.face.FisherFaceRecognizer_create()

model.train(images, labels)

webcam = cv2.VideoCapture(0)
cnt=0

while True:
    (_, im) = webcam.read()
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in faces:
        cv2.rectangle(im,(x,y),(x+w,y+h),(255,255,0),2)
        face = gray[y:y + h, x:x + w]
        face_resize = cv2.resize(face, (width, height))

        prediction = model.predict(face_resize)
        cv2.rectangle(im, (x, y), (x + w, y + h), (0, 255, 0), 3)
        if prediction[1]<800:
            cv2.putText(im,'%s - %.0f' % (names[prediction[0]],prediction[1]),(x-10, y-10), cv2.FONT_HERSHEY_PLAIN,2,(0, 0, 255))
            print (names[prediction[0]])
            cnt=0
        else:
            cnt+=1
            cv2.putText(im,'Unknown',(x-10, y-10), cv2.FONT_HERSHEY_PLAIN,1,(0, 255, 0))
            if(cnt>100):
                print("Unknown Person")
                cv2.imwrite("unKnown.jpg",im)
                cnt=0
    cv2.imshow('FaceRecognition', im)
    key = cv2.waitKey(10)
    if key == 27:
        break

webcam.release()
cv2.destroyAllWindows()





Training...
[[[106 104 101 ...  18  19  17]
  [111 108 104 ...  13  13  12]
  [110 106 100 ...  11  11  11]
  ...
  [133 133 133 ... 143 134 119]
  [133 133 133 ... 149 139 123]
  [133 133 133 ... 153 141 117]]

 [[ 69  54  38 ...  12  13  13]
  [ 72  55  35 ...  11  11  11]
  [ 69  52  31 ...  10  10  10]
  ...
  [142 142 143 ... 128 135 145]
  [143 143 143 ... 132 135 145]
  [143 143 144 ... 134 137 148]]

 [[ 77  67  53 ...  14  15  16]
  [ 87  75  54 ...  10  11  12]
  [ 88  75  52 ...   8  10  10]
  ...
  [144 144 142 ... 131 134 142]
  [144 144 142 ... 132 136 145]
  [144 144 143 ... 134 138 149]]

 ...

 [[216 214 214 ... 213 218 221]
  [217 216 216 ... 209 217 221]
  [218 216 216 ... 208 215 222]
  ...
  [215 215 215 ... 216 215 215]
  [215 215 214 ... 215 215 216]
  [215 214 213 ... 216 216 216]]

 [[219 219 219 ... 206 211 216]
  [218 218 218 ... 204 210 215]
  [218 218 218 ... 200 207 214]
  ...
  [216 215 215 ... 215 215 214]
  [217 216 215 ... 215 215 214]
  [217 216 215 .

KeyboardInterrupt: 